In [1]:
import json
import datetime
import time
import pandas as pd
from pandas import json_normalize
import plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

from requests_html import HTMLSession
session = HTMLSession()

In [2]:
def str_to_timestamp(date):
    obj_datetime = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return datetime_to_timestamp(obj_datetime)

def datetime_to_timestamp(dt):
    return int(time.mktime(dt.timetuple()))

def transform_ohlc(data):
    result = {}
    if 't' in data.keys():
        result['Date'] = data.pop('t')
        for i, v in enumerate(result['Date']):
            result['Date'][i] = datetime.datetime.fromtimestamp(v)
    if 'o' in data.keys():
        result['Open'] = data.pop('o')
    if 'h' in data.keys():
        result['High'] = data.pop('h')
    if 'l' in data.keys():
        result['Low'] = data.pop('l')
    if 'c' in data.keys():
        result['Close'] = data.pop('c')
    if 'v' in data.keys():
        result['Volume'] = data.pop('v')
        
    df = pd.DataFrame.from_dict(result)
    return df.set_index('Date')


host = 'https://tvc4.forexpros.com/bf9b0cf35ce6ca91b1bba8dd1d70c0c2/1601606035/18/18/88/'

def history(symbol, resolution, from_date, to_date=None):
    from_timestamp = str_to_timestamp(from_date + ' 00:00:00' if len(from_date) == 10 else from_date)
    if not to_date:
        to_timestamp = datetime_to_timestamp(datetime.datetime.now())
    else:     
        to_timestamp = str_to_timestamp(to_date + ' 23:59:59' if len(to_date) == 10 else to_date)

    uri = f'history?symbol={symbol}&resolution={resolution}&from={from_timestamp}&to={to_timestamp}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)
    return transform_ohlc(data)

def quotes(symbols):
    symbols_text = ','.join(symbols)
    uri = f'quotes?symbols={symbols_text}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)
    d = [row['v'] for row in data['d']]
    return pd.DataFrame(d)

def symbols(symbol):
    uri = f'symbols?symbol={symbol}'
    r = session.get(host + uri)
    data = json.loads(r.html.text)    
    return json_normalize(data)

def search(query, limit=30):
    uri = f'search?limit={limit}&query={query}&type=&exchange='
    r = session.get(host + uri)
    data = json.loads(r.html.text)    
    return pd.DataFrame(data)
    

In [63]:
df = history(43433, 'D', '2020-01-01')

In [64]:
df['Close'].iplot(kind='line')

In [24]:
지수 = ['KS:KS11', 'KS:KS200', 'KQ:KQ11', 'T:JP225', 'AX:AXJ0', 'HK:HK50', 'SS:FTXIN9', 'NYSE:DJI', 'NYSE:US500',
     'NASDAQ:IXIC', 'P:US2000', 'DE:DE30', 'LON:UK100', 'DE:STOXX50', 'PA:F40', 'MI:IT40', 'TW:TWII', 'KA:KSE',
     'CM:CSE', 'BK:SETI']
주식 = ['NASDAQ:AAPL', 'NYSE:BABA', 'NYSE:TWTR', 'NYSE:AA', 'NYSE:BAC', 'NYSE:KO', 'NYSE:XOM', 'NASDAQ:FB',
     'NASDAQ:GOOGL', 'NYSE:HPQ', 'NYSE:JPM', 'NYSE:MCD', 'NASDAQ:MSFT', 'NASDAQ:SBUX']
선물 = ['COMEX:GC', 'NYMEX:CL', 'COMEX:SI', 'NYMEX:NG', 'CBOT:ZW', 'COMEX:HG', 'KS:KSU7', 'HK:HCEIc1', 'OSE:JP225',
     'Eurex:DE30', 'CME:US500', 'CME:USTEC']
외환 = ['USD/KRW', 'USD/JPY', 'JPY/KRW', 'EUR/USD', 'AUD/USD', 'itBit:BTC/USD', 'ICE:DX', 'EUR/KRW', 'GBP/USD',
     'EUR/JPY', 'USD/CHF', 'USD/CAD', 'GBP/JPY', 'NZD/USD', 'AUD/JPY', 'USD/CNY', 'MCX:USD/RUB']
종합 = ['COMEX:GC', 'COMEX:SI', 'NYMEX:CL', 'NYMEX:NG', 'NYMEX:USTEC', 'CME:USTEC', 'CME:US500', 
        'CBOT:US30', 'CBOT:ZS', 'CBOT:ZW', 'CBOE:VX', 'Eurex:DE30', 'OSE:JP225', 'KRX:KS', '서울:005930', 
        'USD/KRW', 'CHF/CAD', 'USD/CHF', 'USD/CAD']

In [25]:
quotes(외환)

,ch,chp,short_name,exchange,description,lp,ask,bid,spread,open_price,high_price,low_price,prev_close_price,volume
0,4.09,0.35,USD/KRW,실시간 FX,USD/KRW,1161.21,"1,161.26","1,161.17",9,"1,157.12","1,162.10","1,156.99",1157.12,"28,722"
1,-0.07,-0.06,USD/JPY,,USD/JPY,105.66,105.67,105.64,3,105.71,105.79,105.62,105.72,"21,032"
2,0.0488,0.45,JPY/KRW,실시간 FX,JPY/KRW,10.9908,10.9917,10.9899,18,10.9420,10.9990,10.9389,10.9420,"49,765"
3,0.0010,0.08,EUR/USD,,EUR/USD,1.1791,1.1793,1.1789,4,1.1780,1.1798,1.1779,1.1781,"18,186"
4,0.0009,0.13,AUD/USD,,AUD/USD,0.7188,0.7189,0.7187,2,0.7180,0.7209,0.7171,0.7179,"19,540"
5,92.5,0.87,BTC/USD,itBit,BTC/USD,10775.8,"10,782.0","10,781.8",25,"10,683.3","10,800.0","10,759.0",10683.2,6
6,-0.105,-0.11,미국 달러 지수,,미국 달러 지수 선물,93.440,93.460,93.420,0,93.453,93.477,93.385,93.545,"1,634"
7,6.07,0.44,EUR/KRW,실시간 FX,EUR/KRW,1369.27,"1,369.35","1,369.18",17,"1,363.20","1,370.19","1,362.99",1363.20,"46,898"
8,0.0011,0.09,GBP/USD,,GBP/USD,1.2989,1.2989,1.2988,1,1.2976,1.3001,1.2972,1.2977,"21,632"
9,0.03,0.02,EUR/JPY,,EUR/JPY,124.58,124.60,124.55,5,124.54,124.70,124.49,124.55,"39,212"


In [19]:
symbols('KS:005930')

,name,exchange-traded,exchange-listed,timezone,minmov,minmov2,pricescale,pointvalue,has_intraday,has_no_volume,volume_precision,ticker,description,type,has_daily,has_weekly_and_monthly,supported_resolutions,intraday_multipliers,session,data_status
0,005930,서울,서울,Asia/Seoul,1,0,1,1,True,False,3,43433,"삼성전자, 한국",Stock,True,True,"[5, 15, 30, 60, 300, D, W, M, 45, 120, 240]","[1, 5, 15, 30, 60, 300]",2;0900-1530:23456,delayed_streaming


In [6]:
search('삼성')

,symbol,full_name,description,type,ticker,exchange
0,005930,서울:005930,삼성전자,주식,43433,서울
1,028260,서울:028260,삼성물산,주식,43477,서울
2,006400,서울:006400,삼성SDI,주식,43450,서울
3,032830,서울:032830,삼성생명,주식,43428,서울
4,000810,서울:000810,삼성화재해상보험,주식,43370,서울
5,207940,서울:207940,삼성바이오로직스,주식,993256,서울
6,018260,서울:018260,삼성에스디에스,주식,948394,서울
7,009150,서울:009150,삼성전기,주식,43350,서울
8,010140,서울:010140,삼성중공업,주식,43366,서울
9,028050,서울:028050,삼성엔지니어링,주식,43435,서울


In [4]:
tickers = {
    8874: {'symbol': 'USTEC', 'description': '나스닥 100 선물'}
}

{'symbol': 'USTEC', 'description': '나스닥 100 선물'}

In [79]:
targets = [
    ['삼성전자', 43433],
    ['SK하이닉스', 43430],
    ['네이버', 43493]
]

In [82]:
df_list = [history(ticker, 'D', '2019-01-01')['Close'] for name, ticker in targets]
df = pd.concat(df_list, axis=1)
df.columns = [name for name, ticker in targets] 
df.tail(10)

,삼성전자,SK하이닉스,네이버
Date,,,
2020-09-18 09:00:00,59300.0,83700.0,298000.0
2020-09-20 09:00:00,59300.0,83700.0,298000.0
2020-09-21 09:00:00,59200.0,84500.0,292000.0
2020-09-22 09:00:00,58200.0,81300.0,284500.0
2020-09-23 09:00:00,58600.0,83600.0,296500.0
2020-09-24 09:00:00,57800.0,84300.0,294000.0
2020-09-25 09:00:00,57900.0,82900.0,295000.0
2020-09-27 09:00:00,57900.0,82900.0,295000.0
2020-09-28 09:00:00,58200.0,82500.0,298500.0


In [81]:
df_plot = df / df.iloc[0] - 1.0
df_plot.iplot()